DATA PREP

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,  GridSearchCV, cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier 
import xgboost as xgb
from scipy import stats
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
import seaborn as sns


data1=pd.read_csv("/content/media prediction and its cost.csv",encoding="latin-1")
type = data1.dtypes

data =shuffle(data1)

x = data.drop("food_category", axis = 1)
y = data[["food_category"]]


cat_cols = ['food_family', 'sales_country', 'marital_status',
            'gender', 'education', 'member_card', 'occupation',
            'houseowner', 'avg. yearly_income', 'store_type', 'store_city',
            'store_state', 'media_type', 'unit_sales(in millions)', 'total_children',
            'avg_cars_at home(approx)', 'num_children_at_home', 'avg_cars_at home(approx).1',
            'recyclable_package', 'low_fat', 'coffee_bar', 'video_store', 'salad_bar',
            'prepared_food', 'florist']

ohe_cols = [col for col in cat_cols if 10 >= data[col].nunique() > 2]
X = pd.get_dummies(x, columns = ohe_cols)

LE =LabelEncoder()

X['food_department'] = LE.fit_transform(X['food_department'])
X['promotion_name'] = LE.fit_transform(X['promotion_name'])
X['brand_name'] = LE.fit_transform(X['brand_name'])

X['media_type'] = LE.fit_transform(X['media_type'])
X['store_city'] = LE.fit_transform(X['store_city'])
X['houseowner'] = LE.fit_transform(X['houseowner'])
X['gender'] = LE.fit_transform(X['gender'])
X['marital_status'] = LE.fit_transform(X['marital_status'])

y['food_category'] = LE.fit_transform(y['food_category'])
type1 = X.dtypes

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.20, random_state=42)





DECİSİON TREE

In [ ]:
def dec_tree(X_train, X_test, Y_train, Y_test):

 model = DecisionTreeClassifier(random_state=17)
 param = {'max_depth': range(1, 10), "min_samples_split": [3, 4]}
 tree = GridSearchCV(model, param, cv=5, n_jobs=-1, verbose=True)
 tree.fit(X_train, Y_train)
 tree.best_params_

 tune = DecisionTreeClassifier(**tree.best_params_).fit(X_train, Y_train)

 y_pred = tune.predict(X_test)

 print(classification_report(Y_test, y_pred))


RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
def random_forest(X_train, X_test, Y_train, Y_test):

    model = RandomForestClassifier(random_state=42)
    param = {
        'n_estimators': [200, 500],
        'max_features': ['auto', 'sqrt', 'log2'],
        'max_depth': [4, 5, 6, 7, 8],
        'criterion': ['gini', 'entropy']
    }

    forest = GridSearchCV(estimator=model, param_grid=param, cv= 5)
    forest.fit(X_train, Y_train)
    forest.best_params_

    tune = RandomForestClassifier(**forest.best_params_).fit(X_train, Y_train)

    y_pred = tune.predict(X_test)

    print(classification_report(Y_test, y_pred))
    print(forest.best_params_)
    

random_forest(X_train, X_test, Y_train, Y_test)



LİGHTGBM

In [ ]:
from lightgbm import LGBMClassifier
from lightgbm import plot_importance
def light_gbm(X_train, X_test, Y_train, Y_test):

    model = LGBMClassifier()
    params = {"learning_rate": [0.01, 0.1],
              "n_estimators": [500, 1000],
              "max_depth": [5, 8, 10],
              "num_leaves": [32, 64, 128],
              "min_data_in_leaf": [100, 1000]}

    lgbm = GridSearchCV(model,params,cv=5,n_jobs=-1,verbose=2).fit(X_train, Y_train)

    lgbm.best_params_

    tune = LGBMClassifier(**lgbm.best_params_).fit(X_train, Y_train)

    y_pred = tune.predict(X_test)

    print(classification_report(Y_test, y_pred))
    print(lgbm.best_params_)


light_gbm(X_train, X_test, Y_train, Y_test)


"""
ı cannot get any answers from that model ı didnt get error but 
ı waited about 5 hours and model is still training
"""

KNN and Naive bayes

In [ ]:
def knn(X_train, X_test, Y_train, Y_test):

    params = {'n_neighbors': np.arange(1, 50)}
    model = KNeighborsClassifier()

    knn = GridSearchCV(model,params,cv=5)
    knn.fit(X_train, Y_train)

    knn.best_params_

    tune = KNeighborsClassifier(**knn.best_params_).fit(X_train, Y_train)

    y_pred = tune.predict(X_test)

    print(classification_report(Y_test, y_pred))
    print(knn.best_params_)

from sklearn.naive_bayes import GaussianNB

def n_bayes(X_train, X_test, Y_train, Y_test):
    nb = GaussianNB()
    nb.fit(X_train, Y_train)

    y_pred = nb.predict(X_test)
    print(classification_report(Y_test, y_pred))



print tree

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier(max_depth= 9, min_samples_split= 3)
clf.fit(X_train, Y_train)
tree.plot_tree(clf);